## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marsa Matruh,EG,31.35,27.25,80.60,69,0,8.05,clear sky
1,1,Uppsala,SE,60.00,17.75,66.20,59,21,10.29,few clouds
2,2,Auki,SB,-8.77,160.70,77.05,89,100,3.69,light rain
3,3,Barrow,US,71.29,-156.79,37.40,100,90,14.99,mist
4,4,Hobart,AU,-42.88,147.33,41.00,86,40,4.70,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marsa Matruh,EG,31.35,27.25,80.60,69,0,8.05,clear sky
2,2,Auki,SB,-8.77,160.70,77.05,89,100,3.69,light rain
5,5,Los Llanos De Aridane,ES,28.66,-17.92,81.00,73,20,17.22,few clouds
7,7,Ribeira Grande,PT,38.52,-28.70,75.20,88,75,6.93,broken clouds
9,9,Veraval,IN,20.90,70.37,81.05,86,100,18.39,moderate rain
11,11,Pedasi,PA,7.53,-80.03,82.71,71,14,7.76,few clouds
14,14,Ajdabiya,LY,30.76,20.23,79.34,69,0,17.22,clear sky
15,15,Georgetown,MY,5.41,100.34,82.40,88,20,3.36,few clouds
16,16,Lalin,ES,42.72,-8.12,80.01,64,20,12.75,few clouds
18,18,Thinadhoo,MV,0.53,72.93,83.80,72,72,8.41,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Marsa Matruh,EG,80.60,clear sky,31.35,27.25,
2,Auki,SB,77.05,light rain,-8.77,160.70,
5,Los Llanos De Aridane,ES,81.00,few clouds,28.66,-17.92,
7,Ribeira Grande,PT,75.20,broken clouds,38.52,-28.70,
9,Veraval,IN,81.05,moderate rain,20.90,70.37,
11,Pedasi,PA,82.71,few clouds,7.53,-80.03,
14,Ajdabiya,LY,79.34,clear sky,30.76,20.23,
15,Georgetown,MY,82.40,few clouds,5.41,100.34,
16,Lalin,ES,80.01,few clouds,42.72,-8.12,
18,Thinadhoo,MV,83.80,broken clouds,0.53,72.93,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# 7. Drop the rows where there is no Hotel Name.
cleaned_hotel_df = hotel_df[hotel_df["Hotel Name"] !='']
cleaned_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Marsa Matruh,EG,80.60,clear sky,31.35,27.25,Beau Site
2,Auki,SB,77.05,light rain,-8.77,160.70,Auki Motel
5,Los Llanos De Aridane,ES,81.00,few clouds,28.66,-17.92,Valle Aridane
7,Ribeira Grande,PT,75.20,broken clouds,38.52,-28.70,Quinta da Meia Eira
9,Veraval,IN,81.05,moderate rain,20.90,70.37,Lords Inn Somnath
...,...,...,...,...,...,...,...
680,Maragogi,BR,78.17,few clouds,-9.01,-35.22,Salinas do Maragogi All Inclusive Resort
681,Tecolutla,MX,89.60,scattered clouds,20.48,-97.02,Hotel Plaza Marsol
683,Bubaque,GW,80.22,overcast clouds,11.28,-15.83,Casa Dora
691,Kudahuvadhoo,MV,84.54,overcast clouds,2.67,72.89,Kudahuvadhoo Retreat


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
cleaned_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt>{Hotel Name}<dd></dd>
<dt>City Name</dt>{City}<dd></dd>
<dt>Country</dt>{Country}<dd></dd>
<dt>Current Weather</dt>{Current Description} and {Max Temp:.1f} °F<dd></dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleaned_hotel_df[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))